**Содержание**<a id='toc0_'></a>    
- [Свёрточный слой (convolution layer)](#toc1_)    
    - [Реализуем функцию, добавляющую padding](#toc1_1_1_)    
- [Код тестирования](#toc2_)    
- [Сравним скорость](#toc3_)    
- [Что внутре у торча в свертке?](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Свёрточный слой (convolution layer)](#toc0_)

Свёрточный слой (convolution layer) – центральное понятие в современном компьютерном зрении. Конечно, в PyTorch есть его реализация. Но всегда (?) полезно, чтобы разобраться, запрограммировать его руками.

In [2]:
import torch

# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])

### <a id='toc1_1_1_'></a>[Реализуем функцию, добавляющую padding](#toc0_)

- возьмем готовую

In [3]:
def get_padding2d(input_images, pad=1):
    padded_images = torch.nn.functional.pad(input_images, pad=(pad, ) * 4)
    
    # варианты
    # padder = torch.nn.ZeroPad2d(pad)
    # padded_images = padder(input_images)

    # s = input_images.shape
    # padded_images = torch.zeros(s[:-2] + (s[-2] + 2*pad, s[-1] + 2*pad))
    # padded_images[:, :, pad:-pad, pad:-pad] += input_images

    # h, w = input_images.shape[-2:]
    # A = torch.cat((torch.zeros(pad, h), torch.eye(w + pad, h)), axis=0)
    # B = torch.cat((torch.zeros(w, pad), torch.eye(w, h + pad)), axis=1)
    # padded_images = A @ input_images.float() @ B

    return padded_images.float()

Сверточный слой это массив фильтров. Каждый фильтр имеет следующую размерность:

- число слоев во входном изображении (для RGB это 3)
- высота фильтра
- ширина фильтра

В ядре (кернеле) все фильтры имеют одинаковые размерность, поэтому ширину и высоту фильтров называют шириной и высотой ядра. Чаще всего ширина ядра равна высоте ядра, в таком случае их называют размером ядра (`kernel_size`).

Также слой имеет такие параметры:

- `padding` - на какое количество пикселей увеличивать входное изображение с каждой стороны.
- `stride` - на сколько пикселей смещается фильтр при вычислении свертки

$L_{out} =⌊\frac{L_{in} +2×padding−dilation×(kernel_{size}−1)−1}{stride} +1⌋$ - для каждого измерения свертки

In [4]:
from math import floor

def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding, dilation=1):
    batch_size, in_channels, h_in, w_in = input_matrix_shape

    h_out = floor((h_in + 2 * padding - dilation * (kernel_size -1) - 1) / stride + 1)
    w_out = floor((w_in + 2 * padding - dilation * (kernel_size -1) - 1) / stride + 1)

    out_shape = [batch_size, out_channels, h_out, w_out]
    return out_shape

calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0)   # [2, 10, 8, 8]

[2, 10, 8, 8]

# <a id='toc2_'></a>[Код тестирования](#toc0_)

In [5]:
import torch
from abc import ABC, abstractmethod


# абстрактный класс для сверточного слоя
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


# класс-обертка над torch.nn.Conv2d для унификации интерфейса
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, dilation=1):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=padding, dilation=dilation, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


# функция, создающая объект класса cls и возвращающая свертку от input_matrix
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


# Функция, тестирующая класс conv2d_cls.
# Возвращает True, если свертка совпадает со сверткой с помощью torch.nn.Conv2d.
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]], #]) 
                         
                        [[[0., 1, 1],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])


    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
              and (custom_conv2d_out.shape == conv2d_out.shape)

print(test_conv2d_layer(Conv2d))

True


In [6]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, **kwargs):
        super().__init__(in_channels, out_channels, kernel_size, stride)
        self.padding = padding

    def __call__(self, input_tensor, bias=None):
        padded_tensor = get_padding2d(input_tensor, pad=self.padding)
        *_, in_height, in_width = padded_tensor.shape

        out_shape = calc_out_shape(padded_tensor.shape, 
                                   self.out_channels, 
                                   self.kernel_size, 
                                   self.stride, 
                                   padding=self.padding, dilation=1)
                                   
        output_tensor = torch.zeros(out_shape)

        for im_idx, image in enumerate(padded_tensor):
            for c_in, in_channel in enumerate(image):
                for c_out in range(self.out_channels):
                    for h_out, h_in in enumerate(range(0, in_height - self.kernel_size + 1, self.stride)):
                        for w_out, w_in in enumerate(range(0, in_width - self.kernel_size + 1, self.stride)):
                            stride = in_channel[h_in:h_in + self.kernel_size, w_in:w_in + self.kernel_size]
                            ker = self.kernel[c_out][c_in]
                            output_tensor[im_idx, c_out, h_out, w_out] += (stride * ker).sum()
        if bias: 
            output_tensor += bias

        return output_tensor

print(test_conv2d_layer(Conv2dLoop))

True


<p>Реализация через циклы очень неэффективна по производительности. Есть целых два способа сделать то же самое с помощью матричного умножения.&nbsp;</p>

<p>На этом шаге будет реализация первым из них.</p>

<p>&nbsp;</p>

<p>Рассмотрим свертку одного одноканального изображения размером 4*4 пикселя (значения пикселей обозначены через X).</p>

<p>Сворачивать будем с ядром из одного фильтра размером 3*3, веса обозначены через W.</p>

<p>Для простоты примем stride = 1.</p>

<p>Тогда выход Y будет иметь размерность 1*1*2*2 (в данном случае на входе одно изображение - это первая единица в размерности, в ядре один фильтр - это вторая единица в размерности выхода).</p>

<p><img alt="" src="https://ucarecdn.com/1845714a-9187-4dca-83ef-6fe44f030391/-/crop/760x275/0,198/-/preview/"></p>

<p>Оказывается, выход свертки можно получить умножением матриц, как показано ниже.</p>

<p>&nbsp;</p>

<p><img alt="" height="339" src="https://ucarecdn.com/e2a38490-d886-47d6-97b0-dc65f8906ba1/" width="776"></p>

<p>&nbsp;</p>

<p><strong>Рекомендуем убедиться в этом, перемножив матрицы на листочке.</strong></p>

<p>Давайте перейдем от простого случая к общему:</p>

<ul>
	<li><strong>Если фильтров в ядре больше одного.</strong> Заметим, что для каждого фильтра, матрица W’ будет умножаться на один и тот же вектор изображения. Значит, можно сконкатенировать матрицы фильтров ядра по вертикали и за одно умножение получить ответ для всех фильтров.</li>
</ul>

<p>&nbsp;</p>

<p><img alt="" height="416" src="https://ucarecdn.com/91757315-13b9-439c-a59d-9fa14629ce52/" width="624"></p>

<p>&nbsp;</p>

<ul>
	<li><strong>Если на входе более одного изображения: </strong>заметим, что матрица W’ одинакова для всех изображений батча, то есть, можно каждое изображение вначале вытянуть в столбец, а затем эти столбцы для всех изображений батча сконкатенировать по горизонтали.</li>
</ul>

<p>&nbsp;</p>

<p><img alt="" height="220" src="https://ucarecdn.com/e8a10b8d-876e-44cb-ad5c-2a56abafa974/" width="681"></p>

<p>&nbsp;</p>

<ul>
	<li><strong>Если в изображении больше одного слоя, </strong>вначале выполним преобразования входа и ядра для каждого слоя, а затем сконкатенируем: вектора разных слоев входа в один большой вектор, а матрицы ядра соответственно в одну длинную матрицу. И мы получим сложение от выходов по слоям в процессе перемножения матриц.</li>
</ul>

<p><strong><img alt="" height="820" src="https://lh5.googleusercontent.com/2M0cSgkwRnEMQ8y2mrnD-D2alEYn3vVsX7UrgRNLV9BbYv6nswIWesOpKjjNpPMgUl0ixOZoUVyeZXHy5Jlfy1bS4lLkrLuo2ZmOH1gYh88aMKgKa_mjrZHAWzYbBtWihg8GDrxK" width="624"></strong></p>

<p>&nbsp;</p>

<p><strong>То есть даже в самом общем случае мы за одно умножение матриц можем получить ответ.</strong></p>

<p>Но рассчитанный таким способом выход не совпадает по размерности с выходом стандартного слоя из PyTorch - нужно изменить размерность.</p>

<p>&nbsp;</p>

<p>В коде уже реализовано:</p>

<ul>
	<li>
	<p>преобразование входного батча изображений</p>
	</li>
	<li>
	<p>умножение матрицы ядра на матрицу входа</p>
	</li>
	<li>
	<p>преобразование ответа</p>
	</li>
</ul>

<p>Напоминание: во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.</p>

<p><strong>Вам осталось реализовать преобразование ядра в описанный выше формат.</strong></p>

<p><strong>Обратите внимание, что в коде рассматривается общий случай - вход состоит из нескольких многослойных изображений, в ядре несколько слоев.</strong></p></span></div>

In [7]:
class Conv2dMatrix(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, dilation=1, **kwargs):
        """негоже без паддинга"""
        super().__init__(in_channels, out_channels, kernel_size, stride)
        self.padding = padding
        self.dilation = dilation

    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        """как делать не надо"""
        *_, ker_size, ker_size = self.kernel.shape
        *_, h, w = torch_input.shape
        
        kernel_unsqueezed = None
        for ch_out_ker in self.kernel:
            ch_out_ker_unsqueeze = None
            
            for ch_in_ker in ch_out_ker:
                # добиваем 0 справа ядро вх.канала до ширины входа, вытягиваем в строку, добиваем 0 до длины вытянутого в строку входа
                s = torch.nn.functional.pad(torch.nn.functional.pad(ch_in_ker, pad=(0,h-ker_size,0,0)).flatten(), pad=(0, h * (h-ker_size)))
                ch_in_ker_unsqueeze = s.unsqueeze(0) # добавляем измерение в начало, чтоб могло стакаться

                for i in range(max(0, h-ker_size*self.stride)):                                         # по количеству страйдов по высоте
                    shifted = torch.roll(s, self.stride*(i+1))                                          # сдвигаем вытянутое ядро на размер страйда
                    ch_in_ker_unsqueeze = torch.cat((ch_in_ker_unsqueeze, shifted.unsqueeze(0)), 0)     # стакаем снизу к предыдущему ядру
                    for j in range(max(0, w-ker_size*self.stride)):                                     # по количеству страйдов по ширине
                        shifted = torch.roll(ch_in_ker_unsqueeze, w*self.stride*(i+1))                  # сдвигаем вытянутое ядро на (размер страйда * ширину)
                        ch_in_ker_unsqueeze = torch.cat((ch_in_ker_unsqueeze, shifted), 0)              # стакаем снизу к предыдущему ядру
                
                # если есть еще канальные ядра, стакаем справа
                ch_out_ker_unsqueeze = ch_in_ker_unsqueeze if ch_out_ker_unsqueeze is None else torch.cat((ch_out_ker_unsqueeze, ch_in_ker_unsqueeze), 1)
            # если в фильтре несколько ядер, стакаем снизу
            kernel_unsqueezed = ch_out_ker_unsqueeze if kernel_unsqueezed is None else torch.cat((kernel_unsqueezed, ch_out_ker_unsqueeze), 0)

        return kernel_unsqueezed

    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        """как примерно это задумывалось"""
        _, in_channels, in_height, in_width = torch_input.shape
        ku_size = [self.out_channels, output_height, output_width, in_channels, in_height, in_width]
        kernel_unsqueezed = torch.zeros(ku_size, dtype=torch.float32)
        for i in range(output_height):
            for j in range(output_width):
                h_slice = slice(i*self.stride, i*self.stride+self.kernel_size)
                w_slice = slice(j*self.stride, j*self.stride+self.kernel_size)
                kernel_unsqueezed[:, i, j, :, h_slice, w_slice] = self.kernel.type(torch.float32)
        return kernel_unsqueezed.view(-1, in_channels*in_height*in_width)

    def __call__(self, torch_input, bias=None):
        batch_size, out_channels, output_height, output_width \
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.out_channels,
                kernel_size=self.kernel_size,
                stride=self.stride,
                padding=self.padding)
        
        torch_input = get_padding2d(torch_input, pad=self.padding)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)

        result = result.permute(1, 0).view((batch_size, self.out_channels,
                                            output_height, output_width))
                                            
        if bias: 
            result += bias

        return result

print(test_conv2d_layer(Conv2dMatrix))

True


<span><p>На прошлом шаге W’ имеет много нулей. Это снижает эффективность метода.</p>

<p>На этом шаге будет реализация через матрицы другим, более эффективным способом.</p>

<p>Пусть в этот раз на входе батч из одного трехслойного (RGB) изображения размером 3*3.</p>

<p>Пусть ядро имеет 2 фильтра шириной и высотой 2 пикселя.</p>

<p>Тогда выход должен иметь размерность 1*2*2*2.</p>

<p>Пусть W - веса ядра, X - значения входной матрицы, Y - значения на выходе.</p>

<p>Для простоты слои изображения и слои фильтров ядра покрашены в цвета.</p>

<p><strong>Обратите внимание</strong>, например, "синяя" X0 не обязано быть равно "красному" X0, аналогично и про значения в фильтрах ядра - разный цвет и одинаковые переменные могут иметь разные значения, такое обозначение выбрано, чтобы не загромождать рисунок сложными индексами.</p>

<p><img alt="" src="https://ucarecdn.com/ddc6ccbe-2aef-4c7b-a47b-a15e67d3f6ec/"></p>

<p>&nbsp;</p>

<p>Если в первом матричном способе мы вытягивали изображения в столбцы, то теперь будем вытягивать фильтры кернела в строки.</p>

<p>&nbsp;</p>

<p><img alt="" src="https://ucarecdn.com/afbc3c3c-a347-4248-b0de-cd614e637fc3/"></p>

<p>&nbsp;</p>

<p><strong>Рекомендуем проверить на листочке, что результат произведения таких матриц дает тот же результат, что и свертка.</strong></p>

<p>Давайте перейдем от простого случая к общему:</p>

<ul>
	<li>
	<p><strong>Если изображений в батче больше одного</strong>: преобразования ядра от этого не меняется, а преобразованные матрицы входных изображений конкатенируются по горизонтали.</p>
	</li>
</ul>

<p>&nbsp;</p>

<p>Но рассчитанный таким способом выход не совпадает по размерности с выходом стандартного слоя из PyTorch - нужно изменить размерность.</p>

<p>&nbsp;</p>

<p>Функция умножения матриц уже реализована.</p>

<p>&nbsp;</p>

<p>Напоминание: во всех шагах этого урока мы считаем bias в сверточных слоях нулевым.</p>

<p><strong>Требуется написать функции преобразования ядра и входа.</strong></p></span>

In [8]:
class Conv2dMatrixV2(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, dilation=1, **kwargs):
        """негоже без паддинга"""
        super().__init__(in_channels, out_channels, kernel_size, stride)
        self.padding = padding
        self.dilation = dilation

    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        # converted_kernel = torch.cat(tuple(self.kernel.flatten(2)), 0).view(self.out_channels, -1)
        converted_kernel = self.kernel.flatten(1)   #  просто вот
        return converted_kernel
    
    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        converted_input=None
        for i in range(output_height):
            for j in range(output_width):
                h_slice = slice(i*self.stride, i*self.stride + self.kernel_size)
                w_slice = slice(j*self.stride, j*self.stride + self.kernel_size)
                fs = torch_input[:, :, h_slice, w_slice].flatten().unsqueeze(0)
                converted_input = fs if converted_input is None else torch.cat((converted_input, fs), 0)
        converted_input = converted_input.transpose(1, 0).reshape(torch_input.shape[0], -1, output_height*output_width)

        return converted_input

    def _convert_input(self, torch_input, output_height, output_width): # сигнатура старая
        *_, input_height, input_width = torch_input.shape

        # индексы элементов, покрытых первым ядром первого канала певого фильтра в 2d матрице данных
        ker_mask = input_height * torch.arange(0, self.kernel_size*self.dilation, self.dilation).unsqueeze(0).T + \
                                  torch.arange(0, self.kernel_size*self.dilation, self.dilation).unsqueeze(0)   
        flat_ker_mask = ker_mask.flatten().unsqueeze(0) # в строку

        # индексы смещений ядра по 2d данным, с учетом параметров свертки
        windows = input_height * torch.arange(0, input_height - self.kernel_size*self.dilation + 1, self.stride).unsqueeze(0).T + \
                                 torch.arange(0, input_width - self.kernel_size*self.dilation + 1, self.stride).unsqueeze(0)
        flat_windows = windows.flatten().unsqueeze(0)   # в строку

        flat_mask = flat_windows + flat_ker_mask.T  # индексы всех элементов, покрытых ядром, в вытянутой в строку матрице 2d данных

        # 2d данные - в строку; выбор данных, покрытых ядром; стакнуть вертикально
        return torch_input.flatten(2)[:, :, flat_mask].flatten(start_dim=1, end_dim=2)

    def __call__(self, torch_input, bias=None):
        batch_size, out_channels, output_height, output_width \
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=self.padding,
                dilation=self.dilation)

        torch_input = get_padding2d(torch_input, pad=self.padding)
        
        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        
        result = conv2d_out_alternative_matrix_v2.view(torch_input.shape[0],
                                                            self.out_channels, 
                                                            output_height,
                                                            output_width)
        if bias: 
            result += bias

        return result

print(test_conv2d_layer(Conv2dMatrixV2))

True


# <a id='toc3_'></a>[Сравним скорость](#toc0_)

- как же я неимоверно крут!

In [9]:
batch_size=32
input_height=256
input_width=256
stride=3
dilation=2
padding=1

kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]],
                         
                        [[[0., 1, 1],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

in_channels = kernel.shape[1]
kernel_size = kernel.shape[-1]
out_channels = kernel.shape[0]

input_tensor = torch.arange(0, batch_size * in_channels *
                               input_height * input_width,
                               out=torch.FloatTensor()) \
               .reshape(batch_size, in_channels, input_height, input_width)

*_, output_height, output_width = calc_out_shape(input_tensor.shape, out_channels, kernel_size, stride, padding, dilation)

In [10]:
%%time
conv2d_torch = Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation)
conv2d_torch.set_kernel(kernel)
res_torch = conv2d_torch(input_tensor)

CPU times: user 31.2 ms, sys: 3.12 ms, total: 34.4 ms
Wall time: 25.4 ms


In [11]:
%%time
conv2d_matrixV2 = Conv2dMatrixV2(in_channels, out_channels, kernel_size, stride, padding, dilation)
conv2d_matrixV2.set_kernel(kernel)
res_matrixV2 = conv2d_matrixV2(input_tensor)

CPU times: user 98.4 ms, sys: 18.5 ms, total: 117 ms
Wall time: 67.7 ms


Всего лишь в 3 раза медленне чисто сишного торча. А я хорош!

In [12]:
assert torch.allclose(res_torch, res_matrixV2) \
              and (res_torch.shape == res_matrixV2.shape)
res_torch.shape, res_matrixV2.shape

(torch.Size([32, 2, 85, 85]), torch.Size([32, 2, 85, 85]))

In [13]:
ker = conv2d_matrixV2._convert_kernel()
ker

tensor([[ 0.,  1.,  0.,  1.,  2.,  1.,  0.,  1.,  0.,  1.,  2.,  1.,  0.,  3.,
          3.,  0.,  1., 10., 10., 11., 12., 13., 14., 15., 16., 17., 18.],
        [ 0.,  1.,  1.,  1.,  2.,  1.,  0.,  1.,  0.,  1.,  2.,  1.,  0.,  3.,
          3.,  0.,  1., 10., 10., 11., 12., 13., 14., 15., 16., 17., 18.]])

In [14]:
ker_mask = input_height * torch.arange(0, kernel_size*dilation, dilation).unsqueeze(0).T + torch.arange(0, kernel_size*dilation, dilation).unsqueeze(0)
flat_ker_mask = ker_mask.flatten().unsqueeze(0)

windows = input_height * torch.arange(0, input_height - kernel_size*dilation+1, stride).unsqueeze(0).T + torch.arange(0, input_width - kernel_size*dilation+1, stride).unsqueeze(0)
flat_windows = windows.flatten().unsqueeze(0)

flat_mask = flat_windows + flat_ker_mask.T

# ker_mask, flat_windows, flat_mask, input_tensor[0, 0]

In [15]:
masked_transform = input_tensor.flatten(2)[:, :, flat_mask].flatten(start_dim=1, end_dim=2)
looped_transform = conv2d_matrixV2._convert_input(input_tensor, output_height, output_width)
torch.allclose(looped_transform, masked_transform)

True

# <a id='toc4_'></a>[Что внутре у торча в свертке?](#toc0_)

класс `torch.nn,Conv2d` это обертка, которая делает паддинг, сохраняет параметры свертки и вызывает Си-функцию `F.conv2d`:

    @overload
    def conv2d(input: Tensor, weight: Tensor, bias: Optional[Tensor]=None, stride: Union[_int, _size]=1, padding: Union[_int, _size]=0, dilation: Union[_int, _size]=1, groups: _int=1) -> Tensor: ...

The entry point into the C++ pytorch code for conv2d is here:
    
    /pytorch/aten/src/ATen/native/Convolution.cpp

    at::Tensor conv2d(
        const Tensor& input, const Tensor& weight, const Tensor& bias,
        IntArrayRef stride, IntArrayRef padding, IntArrayRef dilation, int64_t groups) {
    return at::convolution(input, weight, bias, stride, padding, dilation,
                            false, {{0, 0}}, groups);
        }


    at::Tensor convolution(
        const Tensor& input, const Tensor& weight, const Tensor& bias,
        IntArrayRef stride, IntArrayRef padding, IntArrayRef dilation,
        bool transposed, IntArrayRef output_padding, int64_t groups) {
    auto& ctx = at::globalContext();
    return at::_convolution(input, weight, bias, stride, padding, dilation,
                            transposed, output_padding, groups,
                            ctx.benchmarkCuDNN(), ctx.deterministicCuDNN(), ctx.userEnabledCuDNN());
        }

...которая тоже обертка для диспетчирезации разных бекендов (cudnn, mkl, ) с разными конфигурациями входов 1д 2д 3д и т.п. параметров свертки.

**Короче раскидано по куче функций и CPP шаблонов/классов. Общий алгоритм не понять могут не только лишь все...**